In [ ]:
import numpy as np
from typing import List, Dict, Tuple
import logging

In [ ]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
class VectorStore:
    def __init__(self):
        self.documents = []
        self.embeddings = []

    def add_document(self, doc: str, embedding: np.ndarray):
        self.documents.append(doc)
        self.embeddings.append(embedding)

    def retrieve(self, query_embedding: np.ndarray, k: int = 3) -> List[Tuple[str, float]]:
        # Simple cosine similarity for retrieval
        similarities = [
            np.dot(query_embedding, emb) / (np.linalg.norm(query_embedding) * np.linalg.norm(emb))
            for emb in self.embeddings
        ]
        # Get top k documents
        top_k_indices = np.argsort(similarities)[-k:][::-1]
        return [(self.documents[i], similarities[i]) for i in top_k_indices]

In [ ]:
class FeedbackStore:
    def __init__(self):
        self.feedback_data = []

    def store_feedback(self, query: str, response: str, feedback: Dict):
        self.feedback_data.append({
            'query': query,
            'response': response,
            'feedback': feedback
        })

    def get_relevant_feedback(self, query: str) -> List[Dict]:
        # Simple relevance check based on query similarity
        return [fb for fb in self.feedback_data if query.lower() in fb['query'].lower()]

In [ ]:
class AdaptiveRAG:
    def __init__(self):
        self.vector_store = VectorStore()
        self.feedback_store = FeedbackStore()
        self.knowledge_base = [
            "Python is a versatile programming language used for web development and data science.",
            "Machine learning is a subset of artificial intelligence that focuses on building systems that learn from data.",
            "RAG combines retrieval and generation for better context-aware responses."
        ]
        # Initialize vector store with dummy embeddings
        for doc in self.knowledge_base:
            self.vector_store.add_document(doc, np.random.rand(100))

    def generate_embedding(self, text: str) -> np.ndarray:
        # Simulate embedding generation (in practice, use a model like BERT)
        return np.random.rand(100)

    def generate_response(self, query: str, context: List[str]) -> str:
        # Simulate LLM response generation
        context_str = "\n".join(context)
        return f"Based on the context:\n{context_str}\n\nAnswer: {query}"

    def refine_response(self, query: str, initial_response: str, feedback: Dict) -> str:
        # Refine response based on feedback
        if feedback.get('is_correct', False):
            return initial_response
        else:
            # Apply simple refinement based on feedback
            correction = feedback.get('correction', '')
            return f"{initial_response}\nRefined with feedback: {correction}"

    def process_query(self, query: str, feedback: Dict = None) -> Tuple[str, List[str]]:
        logger.info(f"Processing query: {query}")

        # Generate query embedding
        query_embedding = self.generate_embedding(query)

        # Retrieve relevant documents
        retrieved_docs = self.vector_store.retrieve(query_embedding)
        context = [doc for doc, _ in retrieved_docs]

        # Check for relevant feedback
        relevant_feedback = self.feedback_store.get_relevant_feedback(query)
        if relevant_feedback:
            logger.info("Applying previous feedback to refine response")
            context.extend([fb['response'] for fb in relevant_feedback])

        # Generate initial response
        response = self.generate_response(query, context)

        # If feedback is provided, refine the response
        if feedback:
            response = self.refine_response(query, response, feedback)
            self.feedback_store.store_feedback(query, response, feedback)

        return response, context

In [ ]:
def main():
    # Initialize Adaptive RAG system
    rag = AdaptiveRAG()

    query = "What is Python used for?"

    # First query without feedback
    response, context = rag.process_query(query)
    print("Initial Response:")
    print(response)

    # Simulate user feedback
    feedback = {
        'is_correct': False,
        'correction': 'Python is also used for automation and scripting.'
    }

    # Process query with feedback
    refined_response, context = rag.process_query(query, feedback)
    print("\nRefined Response with Feedback:")
    print(refined_response)

In [ ]:
if __name__ == "__main__":
    main()

Initial Response:
Based on the context:
RAG combines retrieval and generation for better context-aware responses.
Python is a versatile programming language used for web development and data science.
Machine learning is a subset of artificial intelligence that focuses on building systems that learn from data.

Answer: What is Python used for?

Refined Response with Feedback:
Based on the context:
RAG combines retrieval and generation for better context-aware responses.
Machine learning is a subset of artificial intelligence that focuses on building systems that learn from data.
Python is a versatile programming language used for web development and data science.

Answer: What is Python used for?
Refined with feedback: Python is also used for automation and scripting.
